Setup
=====

Make sure you have ixdat installed.
This tutorial requires ixdat version 0.2.3 or higher.
ixdat 0.2.3 is available as a pre-release using:

  `pip install ixdat==0.2.3.dev1`

To make sure that we've installed ixdat correctly, we import it in this first line.
Ixdat is nice and tells us where it is being imported from.

Jupyter Notebooks are nice for tutorials, because you can combine text and code and easily run one cell at a time.
However, there are many ways to run python scripts. Two that I particular like are the IDE's (integrated development environments) called *spyder* and *PyCharm*. Spyder comes with *Anaconda*. PyCharm is not, but is also available for free installation. Both of those IDE's can give you interractive plot windows which are nice for zooming. I use spyder for simple plotting and analysis scripts and pycharm if I'm developing something (like working on ixdat or a project repository).

For .py versions of these demonstrations, see here: https://github.com/ixdat/tutorials/tree/main/demos/22E06_ICL

In [ ]:
import ixdat  
# should print "importing ixdat v0.2.3dev1 from ...\Anaconda3\lib\site-packages\ixdat\__init__.py

This next block of code is just defining the folder where your data is. 
We'll use the `pathlib` module of the `Path` package. This is a nice way to keep track of locations on your computer in python. It enables you for example, to use the `/` character to mean "in a folder".

In [ ]:
from pathlib import Path

data_dir = Path("./data")  # this is where the data is. In this case in the "data" subfolder in the folder with this script

# To make sure that we have the right folder, we can print its contents:
for p in data_dir.iterdir():
    print(p)

# it should print the names of the folders "01", "02", "03", and "04"

Demo 1: Plotting EC cycles
======================

This example shows how to read in and plot EC data.
The data is from an iridium oxide electrode which was cycled from the hydrogen region to the oxygen region over night, making a big feature associated (probably) with getting protons in and out of a hydrated layer.

In [ ]:
# In ixdat, you can do most things by starting with the "Measurement" class. Import it like this:
from ixdat import Measurement 

# This loads the EC data:
ec = Measurement.read(
    data_dir / "01/iridium_butterfly_short_CVA.mpt", reader="biologic", name="cool stuff"
) 
# We can see what it is like this:
print(ec)

ECMeasurement
----------------

Every measurement in ixdat has a standard "plotter", and a standard "plot" function.
For an `ECMeasurement`, the standard plot function plots current and potential on left and right y-axes, respectively, against time on the x-axes.

In [ ]:
ec.plot()

So, it's a really long measurement and you can't quite resolve it on this scale. So we should zoom in.

There are a lot of ways to customize the plots output by ixdat. You can check which ones are available for your plotter by using python's help function (this prints the docstring):

In [ ]:
help(ec.plot)

This reveals that one way to zoom in is to use the `tspan` argument.
Here, we'll zoom in, and, for fun, change the color of the current:

In [ ]:
ec.plot(tspan=[1000, 1100], J_color="green")

It'd be nice if it gave the potential vs RHE and the calibrated current.
We can do so using the `calibrate` function. 

(This measurement was done with an HgSO4 RE at pH=1, on a 5-mm-diameter disk electrode, with the WE connected through a 500 Ohm resistor.)

In [ ]:
ec.calibrate(RE_vs_RHE=0.715, A_el=0.196, R_Ohm=500) 

axes = ec.plot(tspan=[1000, 1100])

axes[0].set_ylabel("Potential vs RHE in V")

If we want data *out* of a `Measurement`, we can use its `grab` function, which returns two vectors - one time and one the selected value. Grabbing also gives you an opportunity to specify a `tspan`.

To get a list of what things can be "grabbed", you can look two places: `series_names` and `aliases`:

In [ ]:
print(ec.series_names)
print(ec.aliases)

For example, to grab the raw potential (which is an alias for `Ewe/V`):

In [ ]:
t, v = ec.grab("raw_potential", tspan=[2000, 2005])
print("This is the time vector:")
print(t) 
print("This is the raw potential vector:")
print(v) 

Unfortunately, these lists are not complete, because they don't include *calibrated* series. In this case, because we have calibrated potential and normalized current, you can also grab `"potential"` and `"current"` to get the calibrated values. We are working on a way to make this clear and transparent for users.

For more details on how to get data out of an `ECMeasurement`, see the tutorial here:
https://github.com/ixdat/tutorials/blob/main/electrochemistry/01_reading_and_using_data.ipynb


CyclicVoltammogram
-------------------

Now, if we want to plot it as a cyclic voltammogram, i.e. potential against current, we can use the `plot_vs_potential` function. But it's nicer to convert it to a `CyclicVoltammogram` object as follows.

In [ ]:
cv = ec.as_cv()
print(cv)

Then the *default* plotting method becomes plotting vs potential:

In [ ]:
cv.plot()

Because there are so many cycles in this case, it's hard to see what is going on.
This can be improved by adding color to the cycles, with the plot_cycles function. Notice the color bar.

In [ ]:
help(cv.plot_cycles)

In [ ]:
cv.plot_cycles(cmap_name="rainbow")

We can also select one cycle at a time by indexing. To compare cycles 1 and 200, for example:

In [ ]:
cv.correct_ohmic_drop(R_Ohm=500 * 0.15)

ax = cv[1].plot(color="k", label="early")
cv[200].plot(color="b", ax=ax, label="late")
ax.legend()

Notice the way the `Axis` object returned by the first plot function is *reused* for the second plot function.

A super handy method that comes with ixdat `CyclicVoltammogram` is `redefine_cycle`, which gives you control over where in the CV the cycle number increments.

For example, to get the cycle to increment at 0.5 V vs RHE in the anodic direction, do this:

In [ ]:
cv.redefine_cycle(start_potential=0.5, redox=True)

You can spot the difference by plotting the cycles again:

In [ ]:
ax = cv[1].plot(color="k", label="early")  # TODO: cv.select_cycle(1, clos_endpoints=True)
cv[200].plot(color="b", ax=ax, label="late")
ax.legend()

For more handy stuff on cyclic voltammogram manipulation and analysis, see here: 
https://github.com/ixdat/tutorials/blob/main/electrochemistry/02_comparing_cycles.ipynb

A final word on this demo is that, because of the way that ixdat uses inheritance for one class to build on another, all of the stuff demonstrated here also works for EC-MS data s. That is to say, you can convert an `ECMSMeasurement` to an `ECMSCyclicVoltammogram` using the `as_cv` method and then select cycles by indexing exactly like here.

Demo 2: Two-dimensional data sets
================================

Here we'll take a quick look at using insitu UV-Vis as an example of Spectroelectrochemistry.
The data is courtesy of Caiwu Liang and is taken on an amorphous IrOx film electrodeposited on FTO glass (in prep.).

Again, we use the `Measurement` class's `read` method to load the data from the raw data files.

In [ ]:
from ixdat import Measurement

sec_meas = Measurement.read(
    data_dir / "02/test-7SEC.csv",
    path_to_ref_spec_file=data_dir / "02/WL.csv",
    path_to_U_J_file=data_dir / "02/test-7_JV.csv",
    scan_rate=1,
    tstamp=1,
    reader="msrh_sec",
)
print(sec_meas)

The default plot method of a `SpectroECMeasurement` puts optical absorption data as a heat map in the top panel and electrochemistry data in the bottom panel, all against a common time axis:

In [ ]:
sec_meas.plot(cmap_name="jet")

Just like an `ECMeasurement`, we can calibrate by setting providing the RE potential on the RHE scale.
Unlike an `ECMeasurement`, we can calibrate the optical absorption data by defining the potential at which to consider it the reference potential. Here we use the start potential, 0.6 V vs RHE.

In [ ]:
sec_meas.calibrate_RE(RE_vs_RHE=0.2)
sec_meas.set_reference_spectrum(V_ref=0.6)

This time, we plot the data vs potential using `plot_vs_potential`, and, to make the onset potential more obvious, apply a log scale to the current axis.

In [ ]:
axes = sec_meas.plot_vs_potential(cmap_name="jet")
axes[1].set_yscale("log")

Notice how much smoother the data is after setting the reference spectrum. It gets bumpy at about 1.5 V vs RHE, which is probably due to bubbles.

A vertical cross-section of the absorption data is a spectrum, and a horizontal cross-section would track the intensity of one wavelength with time. As an example of the former:


In [ ]:
spectrum_1 = sec_meas.get_dOD_spectrum(V=1.3, V_ref=0.6)
print(spectrum_1)

It returns a `Spectrum` object, the default plot of which is naturally to plot the spectrum:

In [ ]:
spectrum_1.plot()

This, like `Measurement`'s plot functions, returns an axis, which can be reused, like this:

In [ ]:
ax = spectrum_1.plot(color="k", label="before onset")

spectrum_2 = sec_meas.get_dOD_spectrum(V=1.5, V_ref=1.3)
spectrum_2.plot(color="r", label="change around onset", ax=ax)
ax.set_ylim(bottom=0)

Finally, `SpectroECMeasurement` has a useful plotting method that shows all the spectra together with a scalebar to indicate potential (a "waterfall" plot):

In [ ]:
sec_meas.plot_waterfall()

For more detailed tools on Spectroelectrochemistry, see:
https://github.com/ixdat/tutorials/blob/main/spectroelectrochemistry/spectroelectrochemistry_demo.ipynb

Demo 3: Calibrating EC-MS data
=========================

For this example, we'll use EC and MS data that is spread across a few files.
First, we load and plot both MS data sets, which were taken right after each other:

In [ ]:
ms_1 = Measurement.read(
    data_dir / "03/2022-04-28 19_18_01 Ruti.tsv",
    reader="zilien",
    technique="MS"
)
ms_1.plot()

ms_2 = Measurement.read(
    data_dir / "03/2022-04-28 23_25_58 Ruti.tsv",
    reader="zilien",
    technique="MS"
)
ms_2.plot()

The main "value proposition" of ixdat is centered around its use of the `+` operator for *appending* measurements (`ec + ec` or `ms + ms`) or *hyphenating* measurements (`ec + ms`).

Here we append the two MS measurements and plot the combined one:

In [ ]:
ms = ms_2 + ms_1
ms.plot()

In [ ]:
ec_1 = Measurement.read_set(data_dir / "03/01",  reader="biologic", suffix=".mpt")
ec_1.plot()

ecms_1 = ec_1 + ms
print(ecms_1)

In [ ]:
ecms_1.plot()

In [ ]:
ec_2 = Measurement.read_set(data_dir / "03/06",  reader="biologic", suffix=".mpt")
ecms_2 = ec_2 + ms
ecms_2.plot()

In [ ]:
O2_M32 = ecms_1.ecms_calibration_curve(
    mol="O2",
    mass="M32",
    n_el=4,
    tspan_list=[(1300, 1350), (1900, 1950), (2500, 2550)],
    tspan_bg=(950, 1000)
)

In [ ]:
print(O2_M32)

In [ ]:
from ixdat.techniques.ms import MSCalResult

CO2_M44 = MSCalResult(mol="CO2", mass="M44", F=O2_M32.F)
print(CO2_M44)

In [ ]:
ecms_2.calibrate(ms_cal_results=[O2_M32, CO2_M44], RE_vs_RHE=0, A_el=0.196)
ecms_2.plot(mol_list=["O2"], tspan=[0, 300], unit="nmol/s", logplot=False, tspan_bg=[50, 80])

In [ ]:
t, n_dot_O2 = ecms_2.grab("n_dot_O2")
print(n_dot_O2)

Demo 4: Normalizing Spectra
=======================

In this tutorial, unlike the last three which started with the `Measurement` class, we will start with the `Specturm` class to read the data.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from ixdat import Spectrum


# This is a dictionary which, for each sample name, has a formula, a file name, and a color:
to_plot = {
    "Ru": ("RuO$_2$", "Rude4_omega0p5.xrdml", "black"),
    "RuTi_1": ("Ru$_{0.9}$Ti$_{0.1}$O$_2$", "RutiZelensky_omega0p5.xrdml", "blue"),
    "RuTi_2": ("Ru$_{0.75}$Ti$_{0.25}$O$_2$", "RutiMacron_omega0p5.xrdml", "green"),
    "Ti": ("TiO$_2$", "Poseidon_omega0p5.xrdml", "red")
}

# We'll import the four spectra one at a time, plot them, and put them in here for later use:
spectra = {}

for name, (formula, file_name, color) in to_plot.items():
    xrd = Spectrum.read(data_dir / "04" / file_name, reader="xrdml")
    ax = xrd.plot(color=color)
    ax.set_title(formula)
    spectra[name] = xrd

In [ ]:
# Now we'll plot them again, but on one axis and normalize to the substrate peak

fig, ax = plt.subplots()
for name, (formula, file_name, color) in to_plot.items():
    xrd = spectra[name]
    x, y = xrd.x, xrd.y
    y_fto_peak = max(y[np.logical_and(37 < x, x < 38.5)])
    ax.plot(x, y / y_fto_peak, label=formula, color=color)

ax.set_xlabel("two theta / [deg]")
ax.set_ylabel("norm. intensity")
ax.legend()